In [3]:
from dotenv import load_dotenv, find_dotenv
from langchain.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolExecutor
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

_ = load_dotenv(find_dotenv())

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)
tool_executor = ToolExecutor([tavily_tool])

In [4]:
import json
from collections import defaultdict
from typing import List

from langchain.schema import AIMessage, BaseMessage
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import ToolInvocation
from chains import parser_json


def execute_tools(state: List[BaseMessage]) -> List[ToolMessage]:
    tool_invocation: AIMessage = state[-1]
    parsed_tool_calls = parser_json.invoke(tool_invocation)

    ids = []
    tool_invocations = []

    for parsed_call in parsed_tool_calls:
        for query in parsed_call["args"]["search_queries"]:
            tool_invocations.append(ToolInvocation(
                tool="tavily_search_results_json",
                tool_input=query
            ))
            ids.append(parsed_call["id"])
    outputs = tool_executor.batch(tool_invocations)

    # Map each output to its corresponding ID and tool input
    outputs_map = defaultdict(dict)
    for id_, output, invocation in zip(ids, outputs, tool_invocations):
        outputs_map[id_][invocation.tool_input] = output
    
    # Convert the mapped outputs to ToolMessage objects
    tool_messages = []
    for id_, mapped_output in outputs_map.items():
        tool_messages.append(ToolMessage(
            content=json.dumps(mapped_output),
            tool_call_id=id_
        ))
    return tool_messages

answer: AI-Powered Security Operations Centers (SOCs) or Autonomous SOCs leverage artificial intelligence and machine learning algorithms to enhance cybersecurity threat detection, response, and mitigation processes. These solutions can autonomously analyze vast amounts of security data, identify anomalies, and respond to threats in real-time, reducing manual intervention and improving overall security posture. They offer proactive threat hunting capabilities, incident response automation, and continuous monitoring to combat sophisticated cyber threats effectively.

Some startups leading in this domain include Darktrace, which raised over $200 million in funding, focusing on AI-driven cybersecurity solutions for threat detection. Another notable startup is Vectra AI, securing $100 million to develop AI-powered cybersecurity solutions for threat detection and response. SentinelOne is also prominent, with a raised capital of $267 million, specializing in autonomous endpoint security and 

In [5]:
from langchain.schema import HumanMessage


human_message = HumanMessage(
    content="""Write about AI-Powered SOC / autonomous SOC problem domain, 
    list startups that do that and raised capital"""
)

In [6]:
from schemas import AnswerQuestion, Reflection


answer = AnswerQuestion(
    answer="",
    reflection=Reflection(missing="", superfluous=""),
    search_queries=[
        "AI-powered SOC startups funding",
        "AI SOC problem domain specifics",
        "Technologies used by AI-powered SOC startups",
    ],
    id="call_KpYHichFFEmLitHFvEhKy1Ra"
)

In [7]:
from langchain.schema import AIMessage


raw_res = execute_tools(
    state=[
        human_message,
        AIMessage(
            content="",
            tool_calls=[
                {
                    "name": AnswerQuestion.__name__,
                    "args": answer.dict(),
                    "id": "call_KpYHichFFEmLitHFvEhKy1Ra"
                }
            ]
        )
    ]
)

In [8]:
print(raw_res)

[ToolMessage(content='{"AI-powered SOC startups funding": [{"url": "https://venturebeat.com/security/crowdstrike-launches-advanced-siem-to-power-the-ai-native-soc-at-rsac-2024/", "content": "Charlotte AI for all Falcon Data: Charlotte AI, CrowdStrike\'s Generative AI security analyst, is now available for Falcon data in Next Gen SIEM. SOC analysts can ask for Falcon data in the ..."}, {"url": "https://www.statista.com/chart/32063/top-8-ai-startups-worldwide-according-to-capital-invested/", "content": "The sole outlier, Mistral AI, a French company founded just a year ago by former Meta and Google DeepMind employees, has already raised $544 million in capital in less than twelve months. Looking ..."}, {"url": "https://www.securityweek.com/radiant-snags-15-million-for-ai-powered-soc-technology/", "content": "November 14, 2023. Radiant Security, a San Francisco startup with ambitious plans to modernize SOC (security operations center) technology with AI co-pilots, has attracted $15 millio